In [2]:
## Import the essential library

import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

In [9]:
# Load the enviornment
load_dotenv()

True

In [10]:
#Get the Twitter bearer token from the .env file

twitter_bearer_token = os.getenv("TWITTER_BEARER_TOKEN")

#Check if retrieved
type(twitter_bearer_token)

str

In [11]:
## Read the ticker symbols from CSV and put it in the dataframe
csvpath = Path("Data/StockPriceData.csv")
df_stock_price_data = pd.read_csv(csvpath, infer_datetime_format=True, parse_dates=True)

#Delete the unnecessary index column from CSV
df_stock_price_data = df_stock_price_data.drop(columns=df_stock_price_data.columns[0])
df_stock_price_data

,Date,Close,Ticker
0,2022-05-05,6.47,ADSE
1,2022-05-06,6.40,ADSE
2,2022-05-09,6.43,ADSE
3,2022-05-10,5.85,ADSE
4,2022-05-11,5.91,ADSE
...,...,...,...
147,2022-05-05,9.60,ZGN
148,2022-05-06,9.52,ZGN
149,2022-05-09,9.22,ZGN
150,2022-05-10,9.24,ZGN


In [12]:
#List of unique tickers
tickers_list = list(set(df_stock_price_data['Ticker']))
tickers_list

['HYB',
 'CGABL',
 'UPH',
 'CRECU',
 'ROCLW',
 'VAL',
 'BNIXW',
 'ZGN',
 'INVH',
 'LVRAU',
 'NAZ',
 'NLSN',
 'INDT',
 'KBNTW',
 'SBT',
 'CLLS',
 'ALF',
 'SVM',
 'ADSE',
 'JUGG',
 'FICV',
 'TNXP',
 'CLVT',
 'TUP',
 'VWEWW',
 'EDSA',
 'PLMIW',
 'FTAIO',
 'MBNKP',
 'GLBL',
 'FNVTW']

In [13]:
# Set the Bearer Token for Authorisation for Twitter API call.
auth_token = "Bearer " + twitter_bearer_token
headers = {"Authorization": auth_token}

## Function to read 100 recent tweets related to ticker and from the date time specified.
def read_100_Tweets(ticker, tweet_date_time):
    ## Function to read 100 recent tweets from the specific dates
    ##  Input : ticker - Stock Ticker
    ##          tweet_date_time - UTC Date/Time Format YYYY-MM-DDTHH:mm:ssZ (ISO 8601/RFC 3339)
    ## 
    ##  Output: List of 100 tweets
    twitter_api_url  = f"https://api.twitter.com/2/tweets/search/recent?max_results=100&query={ticker}&start_time={tweet_date_time}"
    response = requests.get(twitter_api_url, headers=headers)
    ## Check for 200 status code which means it was successful
    tweets_list = [];
    if(response.status_code == 200):
        json_response = response.json()
        #Check if there are any tweets at all
        if('data' in json_response.keys()): 
            all_tweets = response.json()["data"]
            for tweet in all_tweets:
                tweets_list.append(tweet["text"])
            
    else:
        print(f"Response code: {response.status_code}.  Error in getting the tweet");
    return tweets_list    
    

In [14]:
## Perform Vader Sentiment Analysis

## Define Sentiment Object for Sentiment Analysis

sentiment_obj = SentimentIntensityAnalyzer()
def perform_sentiment_analysis(tweets_list):
    ## Function to read 100 recent tweets from the specific dates
    ##  Input : tweets_list - List of 100 tweets
    ##          
    ## 
    ##  Output: sentiment score average
    
    ##Check if there are tweets to analyse
    if (len(tweets_list) > 0):
        sentiment_scores_all = []
        for tweet in tweets_list:
            sentiment_dict = sentiment_obj.polarity_scores(tweet)
            sentiment_scores_all.append(sentiment_dict["compound"])

        #Average the sentiment of all tweets    
        average_sentiment = np.average(sentiment_scores_all)
        return average_sentiment
    else:
        return 0;
    

    

In [62]:
#perform_sentiment_analysis(tweets_list)

In [15]:
start_date = datetime(2022,5,8, 13,0,0)
end_date = datetime(2022,5,14, 13,0,0)
tweet_sentiments = []

while start_date <= end_date:
    delta = timedelta(days=1)
    for ticker in tickers_list:
        print(f"Executing Tweet Analysis for {ticker} on {start_date.isoformat()}")
        tweets_list = read_100_Tweets(ticker, start_date.isoformat() + "Z")
        sentiment_score = perform_sentiment_analysis(tweets_list)
        tweet_sentiment = {}
        tweet_sentiment["Ticker"] = ticker
        tweet_sentiment["Date"] = start_date
        tweet_sentiment["Sentiment_Score"] = sentiment_score
        tweet_sentiments.append(tweet_sentiment)
        #print("Ticker: " + tweet_sentiment["ticker"] + ", Date : " + str(tweet_sentiment["date"]) + ", Sentiment Score: " + str(tweet_sentiment["sentiment_score"]))
    start_date += delta
 

Executing Tweet Analysis for HYB on 2022-05-08T13:00:00
Executing Tweet Analysis for CGABL on 2022-05-08T13:00:00
Executing Tweet Analysis for UPH on 2022-05-08T13:00:00
Executing Tweet Analysis for CRECU on 2022-05-08T13:00:00
Executing Tweet Analysis for ROCLW on 2022-05-08T13:00:00
Executing Tweet Analysis for VAL on 2022-05-08T13:00:00
Executing Tweet Analysis for BNIXW on 2022-05-08T13:00:00
Executing Tweet Analysis for ZGN on 2022-05-08T13:00:00
Executing Tweet Analysis for INVH on 2022-05-08T13:00:00
Executing Tweet Analysis for LVRAU on 2022-05-08T13:00:00
Executing Tweet Analysis for NAZ on 2022-05-08T13:00:00
Executing Tweet Analysis for NLSN on 2022-05-08T13:00:00
Executing Tweet Analysis for INDT on 2022-05-08T13:00:00
Executing Tweet Analysis for KBNTW on 2022-05-08T13:00:00
Executing Tweet Analysis for SBT on 2022-05-08T13:00:00
Executing Tweet Analysis for CLLS on 2022-05-08T13:00:00
Executing Tweet Analysis for ALF on 2022-05-08T13:00:00
Executing Tweet Analysis for SVM

In [16]:
start_date.isoformat()

'2022-05-15T13:00:00'

In [17]:
   
tweet_sentiments

[{'Ticker': 'HYB',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': 0.10293199999999998},
 {'Ticker': 'CGABL',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': -0.09106666666666667},
 {'Ticker': 'UPH',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': 0.06288181818181819},
 {'Ticker': 'CRECU',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': 0},
 {'Ticker': 'ROCLW',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': 0},
 {'Ticker': 'VAL',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': 0.100846},
 {'Ticker': 'BNIXW',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': 0.7480333333333333},
 {'Ticker': 'ZGN',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': -0.139474},
 {'Ticker': 'INVH',
  'Date': datetime.datetime(2022, 5, 8, 13, 0),
  'Sentiment_Score': 0.06265899999999999},
 {'Ticker': 'LVRAU',
  'Date': datetime.datetime(2022, 5, 

In [18]:
stock_tweet_sentiment_df = pd.DataFrame(tweet_sentiments)
#Date Clean up
stock_tweet_sentiment_df["Date"] = pd.to_datetime(stock_tweet_sentiment_df["Date"].dt.date)
#Check Date Type
stock_tweet_sentiment_df.dtypes

Ticker                     object
Date               datetime64[ns]
Sentiment_Score           float64
dtype: object

In [19]:
stock_tweet_sentiment_df

,Ticker,Date,Sentiment_Score
0,HYB,2022-05-08,0.102932
1,CGABL,2022-05-08,-0.091067
2,UPH,2022-05-08,0.062882
3,CRECU,2022-05-08,0.000000
4,ROCLW,2022-05-08,0.000000
...,...,...,...
212,PLMIW,2022-05-14,0.000000
213,FTAIO,2022-05-14,0.000000
214,MBNKP,2022-05-14,0.000000
215,GLBL,2022-05-14,0.264156


In [20]:
#Sort the tweets df, reset the index after sorting and then drop the column "index"
stock_tweet_sentiment_df = stock_tweet_sentiment_df.sort_values(by=["Ticker", "Date"]).reset_index().drop(columns="index")
stock_tweet_sentiment_df

,Ticker,Date,Sentiment_Score
0,ADSE,2022-05-08,0.333159
1,ADSE,2022-05-09,0.333159
2,ADSE,2022-05-10,0.333159
3,ADSE,2022-05-11,0.333159
4,ADSE,2022-05-12,0.333159
...,...,...,...
212,ZGN,2022-05-10,-0.139474
213,ZGN,2022-05-11,-0.139474
214,ZGN,2022-05-12,-0.139474
215,ZGN,2022-05-13,-0.133480


In [21]:
stock_tweet_sentiment_df.to_csv("Data/stock_tweet_sentiment.csv", mode="a", header="false")

In [23]:
df_stock_price_data

,Date,Close,Ticker
0,2022-05-05,6.47,ADSE
1,2022-05-06,6.40,ADSE
2,2022-05-09,6.43,ADSE
3,2022-05-10,5.85,ADSE
4,2022-05-11,5.91,ADSE
...,...,...,...
147,2022-05-05,9.60,ZGN
148,2022-05-06,9.52,ZGN
149,2022-05-09,9.22,ZGN
150,2022-05-10,9.24,ZGN


,Date,Close,Ticker
0,2022-05-05,6.47,ADSE
1,2022-05-06,6.40,ADSE
2,2022-05-09,6.43,ADSE
3,2022-05-10,5.85,ADSE
4,2022-05-11,5.91,ADSE
...,...,...,...
147,2022-05-05,9.60,ZGN
148,2022-05-06,9.52,ZGN
149,2022-05-09,9.22,ZGN
150,2022-05-10,9.24,ZGN
